In [ ]:
!pip install  langchain_community
!pip install unstructured
!pip install "unstructured[docx]"
!pip install langchain_chroma
!pip install -U langchain-ollama
!pip install langchain_core

In [1]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("./data", glob="**/*.docx") # 
data = loader.load()
len(data)

1

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)
len(all_splits)

70

In [3]:
# model = "llama3.2:latest"
# model = "all-minilm:latest"
model = "llama3.2:1b-instruct-q2_K"

In [4]:
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings #  The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0.
#from langchain_ollama import OllamaEmbeddings

vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=OllamaEmbeddings(model=model, show_progress=True),
    persist_directory="./chroma_db",
)

C:\Users\ramezania\AppData\Local\Temp\ipykernel_4512\3767905915.py:7: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding=OllamaEmbeddings(model=model, show_progress=True),
OllamaEmbeddings: 100%|████████████████████████████████████████████████████████████████| 70/70 [01:28<00:00,  1.26s/it]


In [5]:
question = "What is Recipe?"
docs = vectorstore.similarity_search(question)
docs

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.83it/s]


[Document(id='5152803e-a221-460f-9ad5-6457dddce157', metadata={'source': 'data\\Concept Palamax AI.docx'}, page_content='Using AI to summarize reports generated by PalaMax or even generate reports using NLP from the data that are shown in Production view page. This can help to streamline the process of production optimization or finding bottlenecks in production.\n\nFuture KPI’s prediction\n\nUsing AI to show possible future KPI’s figures based on previous values. This can help to predict future production or help customer to plan production more effectively.\n\n\n\nExtensions\n\nProcess Optimization:'),
 Document(id='ccb14a5a-f0ef-4131-b75f-8c7a0d2eae41', metadata={'source': 'data\\Concept Palamax AI.docx'}, page_content='SpaCy is one of the newer open-source NLP processing libraries. This Python library performs quickly and is well-documented. It is able to handle large datasets and provides users with a plethora of pre-trained NLP models. SpaCy is geared toward those who are getting

In [6]:
from langchain import hub
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

llm = Ollama(model=model)# "all-minilm:latest\" does not support generate"

retriever = vectorstore.as_retriever()


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_prompt = hub.pull("rlm/rag-prompt")
qa_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

C:\Users\ramezania\AppData\Local\Temp\ipykernel_4512\2610587660.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model=model)# "all-minilm:latest\" does not support generate"
C:\Development\Python311\Lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [12]:
question = "What is the story of 'Gené is Missing' book?"
qa_chain.invoke(question)
qa_chain

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 56.67it/s]


'I don\'t have any information on a book called "Gené is Missing". The provided context appears to be related to an artificial intelligence (AI) smart factory, not a book.'

In [7]:
question = "Who is Arslan?"
qa_chain.invoke(question)
qa_chain

OllamaEmbeddings: 100%|██████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.17it/s]


{
  context: VectorStoreRetriever(tags=['Chroma', 'OllamaEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x000001D083883DD0>, search_kwargs={})
           | RunnableLambda(format_docs),
  question: RunnablePassthrough()
}
| ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])
| Ollama(model='llama3.2:1b-instr